In [11]:
import librosa as lb
import librosa.display
import librosa.feature
import librosa.util
import librosa.effects

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import wave 
import soundfile as sf
import pyaudio

import os

import torch
import torch.nn as nn


In [6]:
def play_wave(wave, wave_info):
    # initialize PyAudio
    p = pyaudio.PyAudio()

    # open a stream
    stream = p.open(format=pyaudio.paFloat32,
                    channels=wave_info.channels,
                    rate=wave_info.samplerate,
                    output=True)

    # play audio
    stream.write(wave.tobytes())

    # stop stream and terminate PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

In [7]:
dir_path = r"C:\Users\llama\Desktop\cuni\bakalarka\data\test\kicks" 
file_paths = [os.path.join(dir_path, path) for path in os.listdir(dir_path)]

In [8]:
class Wave():
    def __init__(self, array, sr, info = None) -> None:
        self.array = array
        self.sr = sr
        self.info = info

waves = []

for path in file_paths:
    array, sr = lb.load(path)
    info = sf.info(path)
    waves.append(Wave(array, sr, info))

In [9]:
def pad_or_trim(mfcc, length = 100):
    if mfcc.shape[1] > length:
        return mfcc[:,:length]
    else:
        return np.pad(mfcc, ((0,0),(0,length-mfcc.shape[1])), constant_values = 0)

In [10]:
mfccs = []

for wave in waves:
    mfcc = lb.feature.mfcc(y=wave.array, sr=wave.sr, n_mfcc=256, n_fft=2048, hop_length=256, lifter=0, dct_type=3, n_mels = 128)
  
    print(f"{mfcc.shape} {wave.info.duration}")
    mfcc_pad_or_trim = pad_or_trim(mfcc, 100)
    print(f"{mfcc_pad_or_trim.shape} {wave.info.duration}")

    mfccs.append(mfcc_pad_or_trim)

    print(f"{mfcc_pad_or_trim[-1][-1]}")
    print("")
    inverse = lb.feature.inverse.mfcc_to_audio(mfcc, sr=wave.sr, n_fft=2048, hop_length=256, lifter=0, dct_type=3, n_mels = 128)

    play_wave(wave.array, wave.info)



(128, 38) 0.4342403628117914
(128, 100) 0.4342403628117914
0.0

(128, 42) 0.48387755102040814
(128, 100) 0.48387755102040814
0.0

(128, 46) 0.5241950113378685
(128, 100) 0.5241950113378685
0.0

(128, 28) 0.3163718820861678
(128, 100) 0.3163718820861678
0.0

(128, 26) 0.30156462585034016
(128, 100) 0.30156462585034016
0.0

(128, 21) 0.23564625850340137
(128, 100) 0.23564625850340137
0.0



KeyboardInterrupt: 

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(1, 8, (5,1))
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.conv1(x)
    
model = Model()

print(mfccs[0].shape)
out = model.forward(torch.Tensor(mfccs[0]).view(1,1,128,100))
print(out.shape)
    

(128, 100)
torch.Size([1, 8, 124, 100])
